## Helper Functions

In [2]:
def remove_extra(x):
    l=x.split('&')
    for i in range(len(l)):
        res=""
        for j in range(len(l[i])):
            if l[i][j] != '(' and l[i][j] != ')':
                res+=l[i][j]
        l[i]='('+res+')'
    x='&'.join(l)
    return x

s='!q|((!r|t)&(!t|r))'
remove_extra(s)

'(!q|!r|t)&(!t|r)'

In [3]:
def simplify(x):
    if x==None:
        return x
    l=x.split('|')
    l1=[]
    l2=[]
    for i in l:
        if len(i) == 1:
            if '!'+i in l:
                l2.append('!'+i)
                continue
        else:
            if i[1] in l:
                l2.append(i[1])
                continue
        if i in l1 or i in l2:
            continue
        else:
            l1.append(i)
    x='|'.join(l1)
    return x
s1="!q|!q|!q|r|q"
simplify(s1)

'r'

## Part 0

# Functions for converting into CNF

In [4]:
def remove_implications(x):
    while '>' in x:
        pos = x.find('>')
        left = x[:pos]
        right = x[pos+1:]
        rem_left=""
        if left[-1] == ')':
            count=0
            for i in range(len(left)-1,-1,-1):
                if left[i] == ')':
                    count+=1
                elif left[i] == '(':
                    count-=1
                if count == 0:
                    rem_left=left[:i]
                    left=left[i:]
                    break
        else:
            rem_left=left[:-1]
            left=left[-1]            
        x= f"{rem_left}!{left}|{right}"
    return x

In [5]:
s1="d|((a&c)>b)"
# s2="a>b"
s3="(a>b)>c"
remove_implications(s1)
# remove_implications(s2)
remove_implications(s3)

'!(!a|b)|c'

In [6]:
def remove_iff(x):
    while '=' in x:
        pos = x.find('=')
        left = x[:pos]
        right = x[pos+1:]
        rem_left=""
        rem_right=""
        if left[-1] == ')':
            count=0
            for i in range(len(left)-1,-1,-1):
                if left[i] == ')':
                    count+=1
                elif left[i] == '(':
                    count-=1
                if count == 0:
                    rem_left=left[:i]
                    left=left[i:]
                    break
        else:
            rem_left=left[:-1]
            left=left[-1]
        if right[0] == '(':
            count=0
            for i in range(len(right)):
                if right[i] == '(':
                    count+=1
                elif right[i] == ')':
                    count-=1
                if count == 0:
                    rem_right=right[i+1:]
                    right=right[:i+1]
                    break
        else:
            rem_right=right[1:]
            right=right[0]
        # print('**************')
        # print(x)
        # print(rem_left)
        # print(left)
        # print(right)
        # print(rem_right)
        x= f"{rem_left}(!{left}|{right})&(!{right}|{left}){rem_right}"
        # print(x)
    return x

In [7]:
s='(R=(!(Q=D)|(P=T)))'
remove_iff(s)

'((!R|(!((!Q|D)&(!D|Q))|((!P|T)&(!T|P))))&(!(!((!Q|D)&(!D|Q))|((!P|T)&(!T|P)))|R))'

In [8]:
def demorgan(x):
    for pos in range(len(x)):
        if x[pos] == '!':
            left = x[:pos]
            right = x[pos+1:]
            rem_right=""
            if right[0] == '(':
                count=0
                for i in range(len(right)):
                    if right[i] == '(':
                        count+=1
                    elif right[i] == ')':
                        count-=1
                    if count == 0:
                        rem_right=right[i+1:]
                        right=right[:i+1]
                        break
                # print(right)
                count=0
                for i in range(1,len(right)):
                    if count == 0:
                        if right[i] == '|':
                            right = '(!'+right[1:i] + '&' + '!'+right[i+1:]
                            break
                        elif right[i] == "&":
                            right = '(!'+right[1:i] + '|' + '!'+right[i+1:]
                            break
                    if right[i] == '(':
                        count+=1
                    elif right[i] == ')':
                        count-=1
                x= f"{left}{right}{rem_right}"
                # print(x)
            else:
                continue
        else:
            continue
    return x

In [9]:
def remove_double_neg(x):
    ret=''
    i=0
    count=0
    while(i<len(x)):
        if x[i] == '!':
            count+=1
            i+=1
            continue
        else:
            if count%2 == 0:
                ret+=x[i]
            else:
                ret+='!'+x[i]
            count=0
            i+=1

    return ret
s='!!a'
print(remove_double_neg(s))

a


In [10]:
s1="!(a|b)"
s2="!(a&b)"
s3="a|!(c|(a&d))"
demorgan(s1)
demorgan(s2)
demorgan(s3)

'a|(!c&(!a|!d))'

In [11]:
x='((!R|(!((!Q|D)&(!D|Q))|((!P|T)&(!T|P))))&(!(!((!Q|D)&(!D|Q))|((!P|T)&(!T|P)))|R))'
x1=demorgan(x)
while x1!=x:
    x=x1
    x1=demorgan(x)
    x1=remove_double_neg(x1)
print(x1)

((!R|(((Q&!D)|(D&!Q))|((!P|T)&(!T|P))))&((((!Q|D)&(!D|Q))&((P&!T)|(T&!P)))|R))


In [12]:
def distribute(x):
    while(True):
        count=0
        for i in range(len(x)-1):
            if x[i]=='(':
                count+=1
            elif x[i]==')':
                count-=1
            if count==0:
                break
        if count==1:
            x=x[1:]
            x=x[:-1]
            # print(x)
        else:
            # print(x)
            break
    count=0
    # print('x',x)
    for ind in range(len(x)):
        # print(count,x[ind])
        if count==0:
            if x[ind]=='&': 
                # print('l',x[:ind])
                # print('r',x[ind+1:])
                l=distribute(x[:ind])
                r=distribute(x[ind+1:])
                res='('+l+')&('+r+')'
                res=remove_extra(x)
                return res
            elif x[ind]=='|':
                # print('l',x[:ind])
                # print('r',x[ind+1:])
                l=distribute(x[:ind])
                r=distribute(x[ind+1:])
                list_l=l.split('&')
                list_r=r.split('&')
                res=''
                for i in list_l:
                    for j in list_r:
                        res=res+'('+i+'|'+j+')'+'&'
                res=res[:-1]
                res=remove_extra(res)
                return res
        if x[ind]=='(':
            # print(count)
            count+=1
        elif x[ind]==')':
            # print(count)
            count-=1
    x=remove_extra(x)
    return x

s1='!q|((!r|t)&(!t|r))'
distribute(s1)

'(!q|!r|t)&(!q|!t|r)'

In [13]:
s1='p|(q&(!r|t))'
distribute(s1)

'(p|q)&(p|!r|t)'

In [14]:
s2='(a|b)&(c|d)'
distribute(s2)

'(a|b)&(c|d)'

In [15]:
s3='p|(((!a|b)&(!b|a))&((c&!d)|(d&!c)))'
distribute(s3)

'(p|!a|b)&(p|!b|a)&(p|c)&(p|!d|d)&(p|!c)'

In [16]:
def cnf(x):
    # print(x)
    x=remove_double_neg(x)
    x=remove_implications(x)
    # print('remove imp',x)
    x=remove_iff(x)
    x=remove_double_neg(x)
    # print('remove iff',x)
    x1=demorgan(x)
    while x1!=x:
        x=x1
        x1=demorgan(x)
        x1=remove_double_neg(x1)
    x=remove_double_neg(x)
    # print('demorgan',x)
    x=distribute(x)
    x=remove_double_neg(x)
    # print('distribute',x)
    return x

s="!((a=b)>(c=d))"
print(cnf(s))

(!a|b)&(!b|a)&(c)&(!d|d)&(!c)


In [17]:
def make_kb(kb):
    kb1=[]
    for x in kb:
        s=cnf(x)
        l=s.split('&')
        for i in l:
            kb1.append(i[1:-1])
    # print(kb1)
    return kb1
            

In [18]:
s=['a>b']
make_kb(s)

['!a|b']

# Function for Resolution

In [19]:
def resolve(x,y):
    lx=x.split('|')
    ly=y.split('|')
    n=len(lx)
    for i in range(len(lx)):
        for j in range(len(ly)):
            if lx[i] == '!'+ly[j] or '!'+lx[i] == ly[j]:
                lx[i]=''
                ly[j]=''
    lx=list(filter(None,lx))
    ly=list(filter(None,ly))
    if len(lx)==0 and len(ly)==0:
        return "Contradiction"
    elif len(lx)==n:
        return None
    res=''
    for i in lx:
        res+=i+'|'
    for i in ly:
        res+=i+'|'
    res=res[:-1]
    return res

s1="a|b"
s2="!a|c"
resolve(s1,s2)

'b|c'

# Part A

## Uninformed Seacrh

In [20]:
def uninformed_search(kb,mode):
    step=0
    kb=make_kb(kb)
    print(kb)
    # for i in range(len(kb)):
    i=0
    n=len(kb)
    while i<n:
        i+=1
        for j in range(i+1,len(kb)):
            res=resolve(kb[i],kb[j])
            if res == "Contradiction":
                step+=1
                print('step:',step,'Contradiction',kb[i],kb[j])
                return True
            elif res != None and res not in kb:
                res=simplify(res)
                kb.append(res)
                if mode:
                    step+=1
                    print('step:',step,'statments used to resolve:',kb[i],kb[j],'New Statement:',res)
        n=len(kb)
    return False

## Greedy Algorithm

In [21]:
def hurestic_selection(kb):
    ret=''
    st1=''
    st2=''
    min_len=100000
    for i in range(len(kb)):
        for j in range(i+1,len(kb)):
            res=resolve(kb[i],kb[j])
            res=simplify(res)
            if res==None:
                continue
            elif res == "Contradiction":
                return "Contradiction",kb[i],kb[j]
            elif len(res) < min_len and res not in kb:
                min_len=len(res)
                ret=res
                st1=kb[i]
                st2=kb[j]
    return ret,st1,st2

In [22]:
def greedy_search(kb,mode):
    kb=make_kb(kb)
    print(kb)
    step=0
    while True:
        res,st1,st2=hurestic_selection(kb)
        if res == "Contradiction":
            step+=1
            print('Step',step,'Contradiction',st1,st2)
            return True
        elif res == '':
            return False
        elif res == None:
            continue
        else:
            kb.append(res)
            if mode:
                step+=1
                print('step:',step,'Statments Used:',st1,st2,'New Statement:',res)
    return False

## Part B

# Testing

In [23]:
def solver_uninformed():
    kb=[]
    s=str(input('Enter n and m: '))
    n,m=map(int,s.split(" "))
    for i in range(n):
        s=input("Enter sentence: ")
        kb.append(s)
    q=input("Enter query: ")
    kb.append('!'+q)
    return uninformed_search(kb,m)


In [24]:
def solver_greedy():
    kb=[]
    s=str(input('Enter n and m: '))
    n,m=map(int,s.split(" "))
    for i in range(n):
        s=input("Enter sentence: ")
        kb.append(s)
    q=input("Enter query: ")
    kb.append('!'+q)
    return greedy_search(kb,m)


In [25]:
def testing(kb,q,m):
    print('KB:',kb)
    print('Query:',q)
    print('Mode:',m)
    kb.append('!'+q)
    print('Uninformed Search:')
    print(uninformed_search(kb,m))
    print('Greedy Search:')
    print(greedy_search(kb,m))
    return

In [26]:
kb1=['p|(q&(r>t))','p>r','q>t','q>(r=t)']
q1='r'
m1=1
testing(kb1,q1,m1)

KB: ['p|(q&(r>t))', 'p>r', 'q>t', 'q>(r=t)']
Query: r
Mode: 1
Uninformed Search:
['p|q', 'p|!r|t', '!p|r', '!q|t', '!q|!r|t', '!q|!t|r', '!r']
step: 1 statments used to resolve: p|!r|t !p|r New Statement: t
step: 2 statments used to resolve: p|!r|t !q|!t|r New Statement: p|!q
step: 3 statments used to resolve: !p|r !q|!r|t New Statement: !p|!q|t
step: 4 statments used to resolve: !p|r !r New Statement: !p
step: 5 statments used to resolve: !p|r p|!q New Statement: r|!q
step: 6 statments used to resolve: !q|t !q|!t|r New Statement: !q|r
step: 7 statments used to resolve: !q|!r|t !q|!t|r New Statement: !q
step: 8 statments used to resolve: !q|!r|t r|!q New Statement: !q|t
step: 9 statments used to resolve: !q|!r|t !q|r New Statement: !q|t
step: 10 statments used to resolve: !q|!t|r !r New Statement: !q|!t
step: 11 statments used to resolve: !q|!t|r !p|!q|t New Statement: !q|r|!p
step: 12 statments used to resolve: !q|!t|r !q|t New Statement: !q|r
step: 13 statments used to resolve: !q|!t

In [27]:
kb2=['!a|!w|!s|!h|c','a','!m|w','m','!m|!o|s','m','o','!e|h','e']
q2='c'
m2=1
testing(kb2,q2,m2)

KB: ['!a|!w|!s|!h|c', 'a', '!m|w', 'm', '!m|!o|s', 'm', 'o', '!e|h', 'e']
Query: c
Mode: 1
Uninformed Search:
['!a|!w|!s|!h|c', 'a', '!m|w', 'm', '!m|!o|s', 'm', 'o', '!e|h', 'e', '!c']
step: 1 statments used to resolve: !m|w m New Statement: w
step: 2 statments used to resolve: m !m|!o|s New Statement: !o|s
step: 3 statments used to resolve: !m|!o|s o New Statement: !m|s
step: 4 statments used to resolve: m !m|s New Statement: s
step: 5 statments used to resolve: !e|h e New Statement: h
False
Greedy Search:
['!a|!w|!s|!h|c', 'a', '!m|w', 'm', '!m|!o|s', 'm', 'o', '!e|h', 'e', '!c']
step: 1 Statments Used: !m|w m New Statement: w
step: 2 Statments Used: !e|h e New Statement: h
step: 3 Statments Used: m !m|!o|s New Statement: !o|s
step: 4 Statments Used: o !o|s New Statement: s
step: 5 Statments Used: !m|!o|s o New Statement: !m|s
step: 6 Statments Used: !a|!w|!s|!h|c a New Statement: !w|!s|!h|c
step: 7 Statments Used: w !w|!s|!h|c New Statement: !s|!h|c
step: 8 Statments Used: h !s|!h|

In [ ]:
# solver_uninformed()
# solver_greedy()